## timeit
> 测量小段Python代码的执行时间

### 模块内容
> timeit 定义了一个公共类Timer,Timer构造函数有两个参数：
- 要测量时间的主语句
- '建立'语句(如初始化变量)

> timeit会运行一次建立语句，然后反复执行主语句，并返回运行主语句花费的累计时间。

### 基本示例

In [1]:
import timeit

# using setitem
t = timeit.Timer("print('main statement')", "print('setup')")

print('TIMEIT:')
print(t.timeit(2))

print('REPEAT:')
print(t.repeat(3, 2))

TIMEIT:
setup
main statement
main statement
0.00020543811842799187
REPEAT:
setup
main statement
main statement
setup
main statement
main statement
setup
main statement
main statement
[0.00018724007532000542, 0.0018099183216691017, 0.0002105897292494774]


### 值存储在字典中
> 例子比较了用不同方法用大量填充一个字典所用的时间，首先使用setup_statement变量初始化一个元组列表，这些元组中包含主语句用来构建字典的字符串和整数。

In [3]:
import timeit
import textwrap
# A few constants
range_size = 1000
count = 1000
setup_statement = ';'.join([
    "l = [(str(x), x) for x in range(1000)]",
    "d = {}",
])

# 工具函数，打印每次迭代花费的时间，然后进一步将这个值缩减为在字典中存储一项所花费的平均时间。
def show_results(result):
    "Print microseconds per pass and per item."
    global count, range_size
    per_pass = 1000000 * (result / count)
    print('{:6.2f} usec/pass'.format(per_pass), end=' ')
    per_item = per_pass / range_size
    print('{:6.2f} usec/item'.format(per_item))


print("{} items".format(range_size))
print("{} iterations".format(count))
print()

# 版本1：__setitem__作为基准，此版本应该是最快的。
# Using __setitem__ without checking for existing values first
print('__setitem__:', end=' ')
t = timeit.Timer(
    textwrap.dedent(
        """
        for s, i in l:
            d[s] = i
        """),
    setup_statement,
)
show_results(t.timeit(number=count))

# 使用setdefaulte来确保字典中已经有的值不会被覆盖
# Using setdefault
print('setdefault :', end=' ')
t = timeit.Timer(
    textwrap.dedent(
        """
        for s, i in l:
            d.setdefault(s, i)
        """),
    setup_statement,
)
show_results(t.timeit(number=count))

# 如果查找现有值产生KeyError异常，这个方法才会增加值
# Using exceptions
print('KeyError   :', end=' ')
t = timeit.Timer(
    textwrap.dedent(
        """
        for s, i in l:
            try:
                existing = d[s]
            except KeyError:
                d[s] = i
        """),
    setup_statement,
)
show_results(t.timeit(number=count))

# 使用in确定字典中是否有某个特定的键
# Using "in"
print('"not in"   :', end=' ')
t = timeit.Timer(
    textwrap.dedent(
        """
        for s, i in l:
            if s not in d:
                d[s] = i
        """),
    setup_statement,
)
show_results(t.timeit(number=count))

1000 items
1000 iterations

__setitem__: 145.54 usec/pass   0.15 usec/item
setdefault : 375.41 usec/pass   0.38 usec/item
KeyError   : 162.32 usec/pass   0.16 usec/item
"not in"   : 129.16 usec/pass   0.13 usec/item


### 从命令行执行
> 除了编程接口，python 还提供了一个命令行接口来测试模块，而不需要自动化测试。